Data extracting

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [2]:
def Scrape(start_page,end_page,file_name): #End page is not inclusive
    names,addresses,beds,baths,year_built,tenure,property_type,mrt,price,sqft = [],[],[],[],[],[],[],[],[],[]
    housing = ['HDB','HDB 2 Rooms','HDB 3 Rooms','HDB 4 Rooms','HDB 5 Rooms','HDB Executive','Condo','Apartment','Walk-up','Landed','Executive Condo','Terraced House','Corner Terrace','Semi-Detached','Bungalow','Good Class Bungalow','Cluster House']
    # chrome_options = webdriver.ChromeOptions()
    for num in range(start_page,end_page): #Iterate through the pages to obtain property listing links
        driver = webdriver.Chrome("chromedriver.exe")
        driver.implicitly_wait(5)
        driver.get('https://www.99.co/singapore/sale?page_num={}'.format(num))
        time.sleep(3)
        links =[]

        soup=BeautifulSoup(driver.page_source, 'html.parser')
        # soup=BeautifulSoup(driver.page_source, 'lxml')
        # soup=BeautifulSoup(driver.page_source, 'xml')

        for link in soup.find_all('a'): 
            if type(link.get('href'))==str and 'Search' in link.get('href') and 'page_num'not in link.get('href'):

                links.append('https://www.99.co'+link.get('href'))


        for c,i in enumerate(links):  #Iterate through the porperty links

            driver.get(i)
            time.sleep(1.5)
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            name = soup.find('h1', attrs={'class': '_3Wogd JMF8h lFqTi _1vzK2'})  #Getting property listing names
            names.append(name.getText())


            address = soup.find_all('p', attrs={'class': 'dniCg _3j72o _2rhE-'}) #Getting property address code
            a = []
            for i in address:
                a.append(i.getText()) #if .getText() doesn't work try to use .text ?
            addresses.append(re.findall('(\(D\d\d\)|\(D\d\))',''.join(a)))

            bed = soup.find_all('p', attrs={'class': '_2sIc2 _29qfj _2rhE-'}) #Getting property number of bed rooms, bathrooms and sqft
            temp1 = ['NA','NA','NA']
            for i in bed:
                if 'Bed'in i.getText().split() or 'Beds' in i.getText().split():
                    temp1[0]=i.getText().split()[0]

                elif 'Bath' in i.getText().split() or 'Baths' in i.getText().split():
                    temp1[1]=i.getText().split()[0]

                elif 'sqft' in i.getText().split():
                    temp1[2]=i.getText().split()[0]
                elif 'Studio' in i.getText().split():
                    temp1[0]=i.getText()            
                else:
                    print('Please maunually get info for page{} and link with index {}'.format(num,c))

            beds.append(temp1[0])
            baths.append(temp1[1])
            sqft.append(temp1[2])

            info = soup.find_all('div', attrs={'class': '_3r4yN XCAFU'}) #Getting porperty tenure, year build and property type
            temp = ['NA','NA','NA']

            for i in info:
                if 'year' in i.getText().lower():
                    temp[0]=i.getText()[:2]

                elif 'Freehold' in i.getText():
                    temp[0]=i.getText()

                elif re.findall('\d\d\d\d',i.getText())!=[]:
                    temp[1]=i.getText()

                elif i.getText() in housing:
                    temp[2]=i.getText()

            tenure.append(temp[0])
            year_built.append(temp[1])
            property_type.append(temp[2])

            # properties = soup.find('a', attrs={'class': '_2aXf0 _2Y9HD'})
            # if properties.getText() in housing:
            #         print(properties.getText())
            #         property_type.append(properties.getText())

            mrts = soup.find_all('p', attrs={'class': '_2sIc2 _2rhE- _1c-pJ'}) #Getting property distance to MRT
            temp2 = ['NA']
            for i in mrts:
                if re.findall('\(\d+ m\)',''.join(i.getText()))!=[]:
                        temp2[0]=re.findall('\(\d+ m\)',''.join(i.getText()))
            mrt.append(temp2[0])
            prices = soup.find('p', attrs={'class': '_1zGm8 _3na6W _1vzK2'}) #Getting property prices
            # print(prices.text[1:])
            price.append(prices.text[1:])

        if len(names)==len(addresses) and len(addresses)==len(beds) and len(beds)==len(baths) and len(baths)==len(year_built) and len(year_built)==len(tenure) and len(tenure)==len(property_type) and len(property_type)==len(mrt) and len(mrt)==len(price):
            print('Houston All good for page N°{} --> Proceed to the next page N°{}!'.format(num, num+1)) #Check that the data are all filled
        else:
            print('shit, houston we have a problem!')  #Break if error
            break
        driver.close()
    data=pd.DataFrame(names,columns=['title'])
    data['address_code']=addresses
    data['beds']=beds
    data['baths']=baths
    data['year_built']=year_built
    data['tenure']=tenure
    data['property_type']=property_type
    data['mrt_distance']=mrt
    data['sqft']=sqft
    data['price']=price
    data.to_csv(file_name)   #Saving as a CSV file
    print('Scraped Successfully! :)')


In [19]:
Scrape(1,11,'Scraped_Data\scraped_pg1_pg10.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\2652237597.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page 1 number Proceed to the next page 2!
Houston All good for page 2 number Proceed to the next page 3!
Houston All good for page 3 number Proceed to the next page 4!
Houston All good for page 4 number Proceed to the next page 5!
Houston All good for page 5 number Proceed to the next page 6!
Houston All good for page 6 number Proceed to the next page 7!
Houston All good for page 7 number Proceed to the next page 8!
Houston All good for page 8 number Proceed to the next page 9!
Houston All good for page 9 number Proceed to the next page 10!
Houston All good for page 10 number Proceed to the next page 11!
Scraped Successfully! :)


In [20]:
df = pd.read_csv('Scraped_Data\scraped_pg1_pg10.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
print(df)

     Unnamed: 0                                                        title  \
0    0           HDB Flat for Sale in 810 Tampines Avenue 4                    
1    1           3 Bed Condo for Sale in Race Course 138                       
2    2           HDB Flat for Sale in 65 Circuit Road                          
3    3           HDB Flat for Sale in 501B Yishun Street 51                    
4    4           1 Bed Condo for Sale in Altez                                 
5    5           5 Bed House for Sale in Belgravia Green                       
6    6           2 Bed Condo for Sale in The Anchorage                         
7    7           HDB Flat for Sale in 204A Punggol Field                       
8    8           HDB Flat for Sale in 411 Tampines Street 41                   
9    9           HDB Flat for Sale in 510 West Coast Drive                     
10   10          3 Bed Condo for Sale in Cape Royale                           
11   11          3 Bed Condo for Sale in

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\3039857813.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [21]:
Scrape(11,21,'Scraped_Data\scraped_pg11_pg20.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\2652237597.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page 11 number Proceed to the next page 12!
Houston All good for page 12 number Proceed to the next page 13!
Houston All good for page 13 number Proceed to the next page 14!
Houston All good for page 14 number Proceed to the next page 15!
Houston All good for page 15 number Proceed to the next page 16!
Houston All good for page 16 number Proceed to the next page 17!
Houston All good for page 17 number Proceed to the next page 18!
Houston All good for page 18 number Proceed to the next page 19!
Houston All good for page 19 number Proceed to the next page 20!
Houston All good for page 20 number Proceed to the next page 21!
Scraped Successfully! :)


In [22]:
Scrape(21,31,'Scraped_Data\scraped_pg21_pg30.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\2652237597.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page 21 number Proceed to the next page 22!
Houston All good for page 22 number Proceed to the next page 23!
Houston All good for page 23 number Proceed to the next page 24!
Houston All good for page 24 number Proceed to the next page 25!
Houston All good for page 25 number Proceed to the next page 26!
Houston All good for page 26 number Proceed to the next page 27!
Houston All good for page 27 number Proceed to the next page 28!
Houston All good for page 28 number Proceed to the next page 29!
Houston All good for page 29 number Proceed to the next page 30!
Houston All good for page 30 number Proceed to the next page 31!
Scraped Successfully! :)


In [23]:
Scrape(31,41,'Scraped_Data\scraped_pg31_pg40.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\2652237597.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page 31 number Proceed to the next page 32!
Houston All good for page 32 number Proceed to the next page 33!
Houston All good for page 33 number Proceed to the next page 34!
Houston All good for page 34 number Proceed to the next page 35!
Houston All good for page 35 number Proceed to the next page 36!
Houston All good for page 36 number Proceed to the next page 37!
Houston All good for page 37 number Proceed to the next page 38!
Houston All good for page 38 number Proceed to the next page 39!
Houston All good for page 39 number Proceed to the next page 40!
Houston All good for page 40 number Proceed to the next page 41!
Scraped Successfully! :)


In [ ]:
Scrape(41,51,'Scraped_Data\scraped_pg41_pg50.csv')

In [53]:
Scrape(51,61,'Scraped_Data\scraped_pg51_pg60.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°51 --> Proceed to the next page N°52!
Houston All good for page N°52 --> Proceed to the next page N°53!
Houston All good for page N°53 --> Proceed to the next page N°54!
Houston All good for page N°54 --> Proceed to the next page N°55!
Houston All good for page N°55 --> Proceed to the next page N°56!
Houston All good for page N°56 --> Proceed to the next page N°57!
Houston All good for page N°57 --> Proceed to the next page N°58!
Houston All good for page N°58 --> Proceed to the next page N°59!
Houston All good for page N°59 --> Proceed to the next page N°60!
Houston All good for page N°60 --> Proceed to the next page N°61!
Scraped Successfully! :)


In [55]:
Scrape(61,71,'Scraped_Data\scraped_pg61_pg70.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°61 --> Proceed to the next page N°62!
Houston All good for page N°62 --> Proceed to the next page N°63!
Houston All good for page N°63 --> Proceed to the next page N°64!
Houston All good for page N°64 --> Proceed to the next page N°65!
Houston All good for page N°65 --> Proceed to the next page N°66!
Houston All good for page N°66 --> Proceed to the next page N°67!
Houston All good for page N°67 --> Proceed to the next page N°68!
Houston All good for page N°68 --> Proceed to the next page N°69!
Houston All good for page N°69 --> Proceed to the next page N°70!
Houston All good for page N°70 --> Proceed to the next page N°71!
Scraped Successfully! :)


In [56]:
Scrape(71,81,'Scraped_Data\scraped_pg71_pg80.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°71 --> Proceed to the next page N°72!
Houston All good for page N°72 --> Proceed to the next page N°73!
Houston All good for page N°73 --> Proceed to the next page N°74!
Houston All good for page N°74 --> Proceed to the next page N°75!
Houston All good for page N°75 --> Proceed to the next page N°76!


ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [57]:
Scrape(81,91,'Scraped_Data\scraped_pg81_pg90.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°81 --> Proceed to the next page N°82!
Houston All good for page N°82 --> Proceed to the next page N°83!
Houston All good for page N°83 --> Proceed to the next page N°84!
Houston All good for page N°84 --> Proceed to the next page N°85!
Houston All good for page N°85 --> Proceed to the next page N°86!
Houston All good for page N°86 --> Proceed to the next page N°87!
Houston All good for page N°87 --> Proceed to the next page N°88!
Houston All good for page N°88 --> Proceed to the next page N°89!
Houston All good for page N°89 --> Proceed to the next page N°90!
Houston All good for page N°90 --> Proceed to the next page N°91!
Scraped Successfully! :)


In [58]:
Scrape(91,101,'Scraped_Data\scraped_pg91_pg100.csv')

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°91 --> Proceed to the next page N°92!
Houston All good for page N°92 --> Proceed to the next page N°93!
Houston All good for page N°93 --> Proceed to the next page N°94!
Houston All good for page N°94 --> Proceed to the next page N°95!
Houston All good for page N°95 --> Proceed to the next page N°96!
Houston All good for page N°96 --> Proceed to the next page N°97!
Houston All good for page N°97 --> Proceed to the next page N°98!
Houston All good for page N°98 --> Proceed to the next page N°99!
Houston All good for page N°99 --> Proceed to the next page N°100!
Houston All good for page N°100 --> Proceed to the next page N°101!
Scraped Successfully! :)


In [59]:
try:
    Scrape(101,111,'Scraped_Data\scraped_pg101_pg111.csv')
except:
    pass

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°101 --> Proceed to the next page N°102!
Houston All good for page N°102 --> Proceed to the next page N°103!
Houston All good for page N°103 --> Proceed to the next page N°104!
Houston All good for page N°104 --> Proceed to the next page N°105!
Houston All good for page N°105 --> Proceed to the next page N°106!
Houston All good for page N°106 --> Proceed to the next page N°107!
Houston All good for page N°107 --> Proceed to the next page N°108!
Houston All good for page N°108 --> Proceed to the next page N°109!
Houston All good for page N°109 --> Proceed to the next page N°110!
Houston All good for page N°110 --> Proceed to the next page N°111!
Scraped Successfully! :)


In [61]:
try:
    Scrape(111,121,'Scraped_Data\scraped_pg111_pg121.csv')
except:
    pass

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°111 --> Proceed to the next page N°112!
Houston All good for page N°112 --> Proceed to the next page N°113!
Houston All good for page N°113 --> Proceed to the next page N°114!
Houston All good for page N°114 --> Proceed to the next page N°115!
Houston All good for page N°115 --> Proceed to the next page N°116!
Houston All good for page N°116 --> Proceed to the next page N°117!
Houston All good for page N°117 --> Proceed to the next page N°118!
Houston All good for page N°118 --> Proceed to the next page N°119!
Houston All good for page N°119 --> Proceed to the next page N°120!
Houston All good for page N°120 --> Proceed to the next page N°121!
Scraped Successfully! :)


In [62]:
try:
    Scrape(121,131,'Scraped_Data\scraped_pg121_pg131.csv')
except:
    pass

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°121 --> Proceed to the next page N°122!
Houston All good for page N°122 --> Proceed to the next page N°123!
Houston All good for page N°123 --> Proceed to the next page N°124!
Houston All good for page N°124 --> Proceed to the next page N°125!
Houston All good for page N°125 --> Proceed to the next page N°126!
Houston All good for page N°126 --> Proceed to the next page N°127!
Houston All good for page N°127 --> Proceed to the next page N°128!
Houston All good for page N°128 --> Proceed to the next page N°129!
Houston All good for page N°129 --> Proceed to the next page N°130!
Houston All good for page N°130 --> Proceed to the next page N°131!
Scraped Successfully! :)


In [63]:
try:
    Scrape(131,141,'Scraped_Data\scraped_pg131_pg141.csv')
except:
    pass

C:\Users\MSI\AppData\Local\Temp\ipykernel_12228\1849449702.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


Houston All good for page N°131 --> Proceed to the next page N°132!
Houston All good for page N°132 --> Proceed to the next page N°133!
Houston All good for page N°133 --> Proceed to the next page N°134!
Houston All good for page N°134 --> Proceed to the next page N°135!
Houston All good for page N°135 --> Proceed to the next page N°136!
Houston All good for page N°136 --> Proceed to the next page N°137!
Houston All good for page N°137 --> Proceed to the next page N°138!
Houston All good for page N°138 --> Proceed to the next page N°139!
Houston All good for page N°139 --> Proceed to the next page N°140!
Houston All good for page N°140 --> Proceed to the next page N°141!
Scraped Successfully! :)


In [ ]:
try:
    Scrape(141,151,'Scraped_Data\scraped_pg141_pg151.csv')
except:
    pass

In [ ]:
try:
    Scrape(151,161,'Scraped_Data\scraped_pg151_pg161.csv')
except:
    pass

In [12]:
try:
    Scrape(161,171,'Scraped_Data\scraped_pg161_pg171.csv')
except:
    pass

In [13]:
try:
    Scrape(171,201,'Scraped_Data\scraped_pg171_pg201.csv')
except:
    pass

In [ ]:
Scrape(201,229,'Scraped_Data\scraped_pg201_pg228.csv')